# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_info_DF = pd.read_csv("../output_data/cities.csv")
city_info_DF


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,esna,25.29,32.55,89.60,33,0,2.24,EG,1603727140
1,cherskiy,68.75,161.30,6.10,93,10,4.90,RU,1603727046
2,mandera,3.94,41.87,87.01,49,100,8.41,KE,1603727140
3,caravelas,-17.71,-39.25,80.47,83,12,7.87,BR,1603727140
4,vaini,-21.20,-175.20,71.60,94,40,9.17,TO,1603727141
...,...,...,...,...,...,...,...,...,...
556,kayerkan,69.35,87.76,3.20,91,40,4.47,RU,1603727531
557,teya,60.38,92.63,17.13,95,19,3.00,RU,1603727531
558,uyuni,-20.46,-66.83,63.41,21,0,13.47,BO,1603727531
559,tual,-5.67,132.75,82.63,71,99,7.23,ID,1603727532


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_info_DF[["Lat","Lng"]]
#locations
humidity = city_info_DF["Humidity"]
#humidity

In [5]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False,max_intensity=100,point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
new_weather_DF = city_info_DF.loc[:,["City","Lat","Lng","Max Temp","Humidity","Wind Speed","Cloudiness","Country","Date"]]
#new_weather_DF
new_weather_DF = new_weather_DF.loc[(new_weather_DF["Max Temp"] < 80)&(new_weather_DF["Max Temp"] > 70)&(new_weather_DF["Wind Speed"] < 10)& (new_weather_DF["Cloudiness"] == 0)]
#new_weather_DF 

new_weather_DF = new_weather_DF.dropna()
new_weather_DF
                                    

,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
54,pimentel,-6.84,-79.93,71.60,64,8.05,0,PE,1603727172
79,bela,25.93,81.98,76.10,30,3.98,0,IN,1603727187
156,myitkyina,25.38,97.40,72.82,82,1.54,0,MM,1603727233
230,san jorge,-31.90,-61.86,78.01,26,4.00,0,AR,1603727305
244,matay,28.42,30.78,77.70,39,9.28,0,EG,1603727307
248,tuni,17.35,82.55,77.41,81,2.53,0,IN,1603727308
259,pathein,16.78,94.73,76.64,90,4.94,0,MM,1603727310
262,syracuse,37.09,15.28,73.40,73,8.05,0,IT,1603727244
263,calama,-22.47,-68.93,75.20,8,2.24,0,CL,1603727311
286,sabha,27.04,14.43,78.78,22,4.50,0,LY,1603727344


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = new_weather_DF.loc[:,["City","Lat","Lng","Max Temp","Humidity","Wind Speed","Cloudiness","Country","Date"]]
#hotel_df
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date,Hotel Name
54,pimentel,-6.84,-79.93,71.60,64,8.05,0,PE,1603727172,
79,bela,25.93,81.98,76.10,30,3.98,0,IN,1603727187,
156,myitkyina,25.38,97.40,72.82,82,1.54,0,MM,1603727233,
230,san jorge,-31.90,-61.86,78.01,26,4.00,0,AR,1603727305,
244,matay,28.42,30.78,77.70,39,9.28,0,EG,1603727307,
248,tuni,17.35,82.55,77.41,81,2.53,0,IN,1603727308,
259,pathein,16.78,94.73,76.64,90,4.94,0,MM,1603727310,
262,syracuse,37.09,15.28,73.40,73,8.05,0,IT,1603727244,
263,calama,-22.47,-68.93,75.20,8,2.24,0,CL,1603727311,
286,sabha,27.04,14.43,78.78,22,4.50,0,LY,1603727344,


In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index,row in hotel_df.iterrows():

    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]

#set up params dict
    params = {
    #"location": target_coordinates,
            "location": f"{lat},{lng}",
            "keyword": target_search,
            "radius": target_radius,
            "type":target_type,
            "key": g_key}
            
    print(f"Retrieveing Results for Index {index}: {city}{lat}{lng}")
    response = requests.get(base_url, params=params)
    response_json = response.json()
    #print(json.dumps(response_json, indent=4))
    results = response_json['results']
          
    try:
        print(f"omg {city}{lat} {results[0]['name']}")
       
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    
    except (KeyError, IndexError):
        print("Missing field/result...skiping.")
              
    print("---------------------------------------------------")

    time.sleep(2)
    
print("done")

Retrieveing Results for Index 54: pimentel-6.84-79.93
omg pimentel-6.84 Hotel Puerto del Sol
---------------------------------------------------
Retrieveing Results for Index 79: bela25.9381.98
omg bela25.93 SPOT ON 46397 Leela Palace
---------------------------------------------------
Retrieveing Results for Index 156: myitkyina25.3897.4
omg myitkyina25.38 Palm Spring Resort Hotel Myitkyina
---------------------------------------------------
Retrieveing Results for Index 230: san jorge-31.9-61.86
omg san jorge-31.9 Hotel Monumental Club San Jorge
---------------------------------------------------
Retrieveing Results for Index 244: matay28.4230.78
omg matay28.42 Shell Authorized Retailer - Al Mostafa
---------------------------------------------------
Retrieveing Results for Index 248: tuni17.3582.55
omg tuni17.35 Sri Maruti Residency
---------------------------------------------------
Retrieveing Results for Index 259: pathein16.7894.73
omg pathein16.78 HOTEL LONKAN
-----------------

In [12]:
#places_data = response.json()
#print(json.dumps(places_data, indent=4,sort_keys=True))
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date,Hotel Name
54,pimentel,-6.84,-79.93,71.60,64,8.05,0,PE,1603727172,Hotel Puerto del Sol
79,bela,25.93,81.98,76.10,30,3.98,0,IN,1603727187,SPOT ON 46397 Leela Palace
156,myitkyina,25.38,97.40,72.82,82,1.54,0,MM,1603727233,Palm Spring Resort Hotel Myitkyina
230,san jorge,-31.90,-61.86,78.01,26,4.00,0,AR,1603727305,Hotel Monumental Club San Jorge
244,matay,28.42,30.78,77.70,39,9.28,0,EG,1603727307,Shell Authorized Retailer - Al Mostafa
248,tuni,17.35,82.55,77.41,81,2.53,0,IN,1603727308,Sri Maruti Residency
259,pathein,16.78,94.73,76.64,90,4.94,0,MM,1603727310,HOTEL LONKAN
262,syracuse,37.09,15.28,73.40,73,8.05,0,IT,1603727244,Re Dionisio Boutique Hotel
263,calama,-22.47,-68.93,75.20,8,2.24,0,CL,1603727311,Hotel Alto del Sol Calama
286,sabha,27.04,14.43,78.78,22,4.50,0,LY,1603727344,فندق علي


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
        info_box_content=hotel_info)

fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))